In [4]:
import requests
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime

# Function to retrieve AQI
def aqi_checker(city_name):
    api_key = "561aeff8e2fc76f3235dffdf50c40671b9a09007"
    url = f"https://api.waqi.info/feed/{city_name}/?token={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        aqi_data = response.json()
        if "data" in aqi_data and "aqi" in aqi_data["data"]:
            return aqi_data['data']['aqi']
    return None

# Function to retrieve weather information
def get_weather(city_name):
    weather_api_key = "fa86cb1d101ed912ad04ddaf49f9395a"
    weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={weather_api_key}&units=metric"

    response = requests.get(weather_url)
    if response.status_code == 200:
        weather_data = response.json()
        temperature = weather_data['main']['temp']
        humidity = weather_data['main']['humidity']
        wind_speed = weather_data['wind']['speed']
        return temperature, humidity, wind_speed
    return None, None, None

# Historical data storage
if "history_data" not in st.session_state:
    st.session_state.history_data = []

# App UI
st.title("🌍 AQI and Weather Checker")

city = st.text_input("Enter City Name")

if st.button("Fetch Data"):
    if city:
        aqi_val = aqi_checker(city)
        temp, humidity, wind = get_weather(city)

        if aqi_val is None or temp is None:
            st.error("Failed to retrieve data. Check the city name or API keys.")
        else:
            # AQI Classification
            if 0 <= aqi_val <= 50:
                classification = "Good"
            elif 51 <= aqi_val <= 100:
                classification = "Moderate"
            elif 101 <= aqi_val <= 200:
                classification = "Unhealthy for Sensitive Groups"
            elif 201 <= aqi_val <= 300:
                classification = "Unhealthy"
            elif 301 <= aqi_val <= 400:
                classification = "Very Unhealthy"
            elif 401 <= aqi_val <= 500:
                classification = "Hazardous"
            else:
                classification = "Unknown"

            # Save history
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            st.session_state.history_data.append(
                {"City": city, "Timestamp": timestamp, "AQI": aqi_val}
            )

            # Display results
            st.success(f"City: {city}")
            st.write(f"AQI: **{aqi_val}** ({classification})")
            st.write(f"🌡 Temperature: {temp} °C")
            st.write(f"💧 Humidity: {humidity}%")
            st.write(f"💨 Wind Speed: {wind} m/s")

# Show AQI Trend
if st.button("Show AQI Trend"):
    df = pd.DataFrame(st.session_state.history_data)
    if not df.empty:
        df["Timestamp"] = pd.to_datetime(df["Timestamp"])
        city_df = df[df["City"] == city]

        if not city_df.empty:
            plt.figure(figsize=(10, 5))
            plt.plot(city_df["Timestamp"], city_df["AQI"], marker="o")
            plt.title(f"AQI Trend for {city}")
            plt.xlabel("Timestamp")
            plt.ylabel("AQI")
            plt.xticks(rotation=45)
            st.pyplot(plt)
        else:
            st.info("No historical data available for this city.")
    else:
        st.info("No data available.")

# Show Heatmap
if st.button("Show AQI Heatmap"):
    df = pd.DataFrame(st.session_state.history_data)
    if not df.empty:
        pivot_table = df.pivot_table(index="City", columns="Timestamp", values="AQI", aggfunc="mean")

        plt.figure(figsize=(10, 6))
        sns.heatmap(pivot_table, annot=True, fmt=".0f", cmap="coolwarm", cbar=True)
        plt.title("AQI Heatmap")
        st.pyplot(plt)
    else:
        st.info("No data available to generate a heatmap.")


2025-09-08 16:35:18.781 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-09-08 16:35:19.529 
  command:

    streamlit run C:\Users\Sunil\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
